In [17]:
from urllib.parse import quote
from datetime import datetime, timedelta
import pprint

import pyotp
import json
import pandas as pd
import requests
import json
import re

# List of possible stock names
# url https://www.nseindia.com/api/master-quote
stock_names = [
    "AARTIIND",
    "ABB",
    "ABBOTINDIA",
    "ABCAPITAL",
    "ABFRL",
    "ACC",
    "ADANIENT",
    "ADANIPORTS",
    "ALKEM",
    "AMBUJACEM",
    "APOLLOHOSP",
    "APOLLOTYRE",
    "ASHOKLEY",
    "ASIANPAINT",
    "ASTRAL",
    "ATUL",
    "AUBANK",
    "AUROPHARMA",
    "AXISBANK",
    "BAJAJ-AUTO",
    "BAJAJFINSV",
    "BAJFINANCE",
    "BALKRISIND",
    "BALRAMCHIN",
    "BANDHANBNK",
    "BANKBARODA",
    "BATAINDIA",
    "BEL",
    "BERGEPAINT",
    "BHARATFORG",
    "BHARTIARTL",
    "BHEL",
    "BIOCON",
    "BOSCHLTD",
    "BPCL",
    "BRITANNIA",
    "BSOFT",
    "CANBK",
    "CANFINHOME",
    "CHAMBLFERT",
    "CHOLAFIN",
    "CIPLA",
    "COALINDIA",
    "COFORGE",
    "COLPAL",
    "CONCOR",
    "COROMANDEL",
    "CROMPTON",
    "CUB",
    "CUMMINSIND",
    "DABUR",
    "DALBHARAT",
    "DEEPAKNTR",
    "DIVISLAB",
    "DIXON",
    "DLF",
    "DRREDDY",
    "EICHERMOT",
    "ESCORTS",
    "EXIDEIND",
    "FEDERALBNK",
    "GAIL",
    "GLENMARK",
    "GMRINFRA",
    "GNFC",
    "GODREJCP",
    "GODREJPROP",
    "GRANULES",
    "GRASIM",
    "GUJGASLTD",
    "HAL",
    "HAVELLS",
    "HCLTECH",
    "HDFCAMC",
    "HDFCBANK",
    "HDFCLIFE",
    "HEROMOTOCO",
    "HINDALCO",
    "HINDCOPPER",
    "HINDPETRO",
    "HINDUNILVR",
    "ICICIBANK",
    "ICICIGI",
    "ICICIPRULI",
    "IDEA",
    "IDFC",
    "IDFCFIRSTB",
    "IEX",
    "IGL",
    "INDHOTEL",
    "INDIACEM",
    "INDIAMART",
    "INDIGO",
    "INDUSINDBK",
    "INDUSTOWER",
    "INFY",
    "IOC",
    "IPCALAB",
    "IRCTC",
    "ITC",
    "JINDALSTEL",
    "JKCEMENT",
    "JSWSTEEL",
    "JUBLFOOD",
    "KOTAKBANK",
    "LALPATHLAB",
    "LAURUSLABS",
    "LICHSGFIN",
    "LT",
    "LTF",
    "LTIM",
    "LTTS",
    "LUPIN",
    "M&M",
    "M&MFIN",
    "MANAPPURAM",
    "MARICO",
    "MARUTI",
    "MCDOWELL-N",
    "MCX",
    "METROPOLIS",
    "MFSL",
    "MGL",
    "MOTHERSON",
    "MPHASIS",
    "MRF",
    "MUTHOOTFIN",
    "NATIONALUM",
    "NAUKRI",
    "NAVINFLUOR",
    "NESTLEIND",
    "NMDC",
    "NTPC",
    "OBEROIRLTY",
    "OFSS",
    "ONGC",
    "PAGEIND",
    "PEL",
    "PERSISTENT",
    "PETRONET",
    "PFC",
    "PIDILITIND",
    "PIIND",
    "PNB",
    "POLYCAB",
    "POWERGRID",
    "PVRINOX",
    "RAMCOCEM",
    "RBLBANK",
    "RECLTD",
    "RELIANCE",
    "SAIL",
    "SBICARD",
    "SBILIFE",
    "SBIN",
    "SHREECEM",
    "SHRIRAMFIN",
    "SIEMENS",
    "SRF",
    "SUNPHARMA",
    "SUNTV",
    "SYNGENE",
    "TATACHEM",
    "TATACOMM",
    "TATACONSUM",
    "TATAMOTORS",
    "TATAPOWER",
    "TATASTEEL",
    "TCS",
    "TECHM",
    "TITAN",
    "TORNTPHARM",
    "TRENT",
    "TVSMOTOR",
    "UBL",
    "ULTRACEMCO",
    "UNITDSPR",
    "UPL",
    "VEDL",
    "VOLTAS",
    "WIPRO",
    "ZYDUSLIFE",
]

nse_holidays_2024 = [
    "2024-01-26",  # Republic Day
    "2024-03-08",  # Mahashivratri
    "2024-03-25",  # Holi
    "2024-03-29",  # Good Friday
    "2024-04-11",  # Id-Ul-Fitr (Ramadan Eid)
    "2024-04-17",  # Shri Ram Navmi
    "2024-05-01",  # Maharashtra Day
    "2024-06-17",  # Bakri Id
    "2024-07-17",  # Moharram
    "2024-08-15",  # Independence Day/Parsi New Year
    "2024-10-02",  # Mahatma Gandhi Jayanti
    "2024-11-01",  # Diwali Laxmi Pujan (Muhurat Trading will be conducted)
    "2024-11-15",  # Gurunanak Jayanti
    "2024-12-25",  # Christmas
    "2024-04-14",  # Dr. Baba Saheb Ambedkar Jayanti (Sunday)
    "2024-04-21",  # Shri Mahavir Jayanti (Sunday)
    "2024-09-07",  # Ganesh Chaturthi (Saturday)
    "2024-10-12",  # Dussehra (Saturday)
    "2024-11-02",  # Diwali-Balipratipada (Saturday)
]
# year = 2024
# month = 7
# end_date = 2024-07-02

res_ticker_df = pd.DataFrame([])
res_candle_stick_df = pd.DataFrame([])

# Load JSON data from a file
with open("NSE.json", "r") as file:
    data = json.load(file)

# Define the regex pattern for names ending with "NSETEST" preceded by numbers
pattern = re.compile(r'\d+NSETEST$')
# Filter objects where "segment" is "NSE_FO"
filtered_data = [
    item
    for item in data
    if item.get("segment") == "NSE_FO"
    and item.get("name")
    not in ["BANKNIFTY", "NIFTY", "FINNIFTY", "MIDCPNIFTY", "NIFTYNXT50"]
    and not pattern.search(item.get("name", ""))
]
print(len(filtered_data))
# Print the filtered data (optional)
# print(json.dumps(filtered_data, indent=2))

# Save the filtered data to a new file
with open("filtered_data.json", "w") as file:
    json.dump(filtered_data, file, indent=2)

23542


In [366]:
def getResponse(url):
    headers = {
        "user-agent": "Chrome/80.0.3987.149 Safari/537.36",
        "accept-language": "en,gu;q=0.9,hi;q=0.8",
        "Accept": "application/json"
    }
    response = requests.get(url, headers=headers, timeout=5)
    return response

In [367]:
def getValidInstrumentTickData(symbolToken, interval='1minute', fromDate='2024-07-01', toDate='2024-07-01'):
    # url = 'https://api.upstox.com/v2/historical-candle/NSE_FO|134606/1minute/2024-07-11/2024-07-1'
    uplinkURL = f'https://api.upstox.com/v2/historical-candle/NSE_FO|{symbolToken}/{interval}/{toDate}/{fromDate}'
    response = getResponse(uplinkURL)
    if response.ok:
        df = pd.DataFrame(response.json()['data']['candles'], columns=['time_stamp','open','high','low','close','volume','open_interest'])
        if not df.empty:
            df.insert(1, 'ticker_id', symbolToken+'-'+fromDate)
            df['cs_id'] = df[['ticker_id', 'time_stamp']].agg('-'.join, axis=1)
            return df

In [368]:
def generate_dates(year, month, holidays, end_date):
    # Get the first and last day of the month
    start_date = datetime(year, month, 1)
    end_date = datetime.strptime(end_date, '%Y-%m-%d')
    
    all_dates = pd.date_range(start=start_date, end=end_date).tolist()

    # Filter out weekends and holidays
    valid_dates = [date for date in all_dates if date.weekday() < 5 and date.strftime('%Y-%m-%d') not in holidays]

    # Format the dates as 'yyyy-mm-dd'
    formatted_dates = [date.strftime('%Y-%m-%d') for date in valid_dates]

    return formatted_dates

In [ ]:
# fnoStocksURL = 'https://www.nseindia.com/api/master-quote'
# response = getResponse(fnoStocksURL)
# print(response.json())

for date in generate_dates(2024, 7, nse_holidays_2024, '2024-07-22'):
    for stock in stock_names:
        print (f'Dumping {stock} on {date}')
        instrumentData = getInstrumentData('NFO', stock)
        # print(instrumentData)
        df = pd.DataFrame(instrumentData['data'], columns=['exchange','tradingsymbol','symboltoken'])
        curr_mon_df = df[df['tradingsymbol'].str.contains("JUL") & ~df['tradingsymbol'].str.contains("FUT")]
        # print(curr_mon_df)
        for symbolToken in curr_mon_df['symboltoken']:
            valid_df = getValidInstrumentTickData(symbolToken, '1minute', date, date)
            res_candle_stick_df = pd.concat([res_candle_stick_df, valid_df])
        # print(curr_mon_df)
        curr_mon_df['tradingsymbol'] = curr_mon_df['tradingsymbol'].apply(lambda x: date+'-'+x)
        # curr_mon_df = curr_mon_df.set_index('tradingsymbol')
        # curr_mon_df.insert(loc=0, column='timeTradingSymbol', value=time_trading_symbol)
        # print(curr_mon_df)
        res_ticker_df = pd.concat([res_ticker_df, curr_mon_df])
        print (f'Done {stock} on {date}')
# print(curr_mon_df.size)
# print(curr_mon_df.iloc[2]['valid_df'])
# print(res_ticker_df)

In [391]:
# print(pd.read_json(res_ticker_df.head(1).iloc[0]['valid_df']))
# print(res_candle_stick_df)
print(res_ticker_df)
print(res_candle_stick_df)

    exchange                        tradingsymbol symboltoken
0        NFO  2024-07-01-ABBOTINDIA25JUL2415500CE       76572
1        NFO  2024-07-01-ABBOTINDIA25JUL2415500PE       76573
2        NFO  2024-07-01-ABBOTINDIA25JUL2416000CE       67235
3        NFO  2024-07-01-ABBOTINDIA25JUL2416000PE       67236
4        NFO  2024-07-01-ABBOTINDIA25JUL2416500CE       49922
..       ...                                  ...         ...
123      NFO    2024-07-01-MANAPPURAM25JUL24310PE       57001
124      NFO    2024-07-01-MANAPPURAM25JUL24315CE       57002
125      NFO    2024-07-01-MANAPPURAM25JUL24315PE       57003
126      NFO    2024-07-01-MANAPPURAM25JUL24320CE       47031
127      NFO    2024-07-01-MANAPPURAM25JUL24320PE       47032

[4523 rows x 3 columns]
                    time_stamp          ticker_id   open   high    low  close  \
0    2024-07-01T15:29:00+05:30   67272-2024-07-01  65.00  65.00  65.00  65.00   
1    2024-07-01T15:28:00+05:30   67272-2024-07-01  65.00  65.00  65.0

In [ ]:
# curr_mon_df.to_pickle('data.pkl')
# curr_mon_df[curr_mon_df.index.duplicated()]
ticker_df=pd.DataFrame(columns=['ticker_id','trade_date','stock_name','expiry_date','strike_price','option_type'])
# print(ticker_df)
ticker_df['ticker_id'] = res_ticker_df ['symboltoken']
ticker_df[['trade_date','stock_name','expiry_date','strike_price','option_type']] = res_ticker_df['tradingsymbol'].apply(extract_components)
ticker_df['ticker_id'] = ticker_df[['ticker_id', 'trade_date']].agg('-'.join, axis=1)
# print(ticker_df)


In [ ]:
# DB ACTIVITY

# Create a PostgreSQL engine
engine = create_engine('postgresql+psycopg2://sd_admin:%s@192.168.1.72:5430/stock-dumps'% quote('sdadmin@postgres'))
print(engine)
# Save the DataFrame to a table named 'optstkdata'
# curr_mon_df.to_sql('optstkdata',if_exists='append', con=engine, index=True)
ticker_df.to_sql('ticker',schema='options',if_exists='append', con=engine, index=False)
res_candle_stick_df.to_sql('candle_stick',schema='options',if_exists='append', con=engine, index=False)